## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import scipy.stats

import matplotlib.pyplot as plt


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


In [3]:
def BandPassFilter(signal,fs):
    '''
    Bandpass Filter with butterworth algorithm
    
    Returns:
        Bandpassed filtered signal
        '''
    pass_band = (40/60.0 , 240/60.0)
    num, den = scipy.signal.butter(3, pass_band, btype = 'bandpass', fs = fs)
    filteredSignal = scipy.signal.filtfilt(num,den,signal)
    return filteredSignal

In [21]:
def FourierTransform(sig, fs):
    '''
    Calculates the Fourier Transform of a signal with sampling frequency fs
    Returns:
        freq and magnitude after fft'''
    freq = np.fft.rfftfreq(4*len(sig),1/fs)
    fft = np.abs(np.fft.rfft(sig, 4*len(sig)))
    return freq, fft

In [22]:
def GetFeatures(ppg, acc, fs):
    '''
    Returs:
    PPG and ACC: Peaks, Peaks frequency , frequency and Magnitude after running fft
    '''
    #Fourier transform for the ppg signal
    ppg_freqs, ppg_fft = FourierTransform(ppg, fs)
    
    #filter data
    ppg_fft[ppg_freqs <=40/60] = 0.0
    ppg_fft[ppg_freqs >= 240/60] = 0.0
    
    #find peaks
    ppg_peaks = sp.signal.find_peaks(ppg_fft, height=200)[0]
    ppg_peaks_f = ppg_freqs[ppg_peaks]
    
    #Similarly for the acc signal
    acc_freqs, acc_fft = FourierTransform(acc, fs)
    
    #filter data
    acc_fft[acc_freqs <= 40/60] = 0.0
    acc_fft[acc_freqs >= 240/60] = 0.0
    
    #find peaks
    acc_peaks = sp.signal.find_peaks(acc_fft , height = None)[0]
    acc_peaks_f = acc_freqs[acc_peaks]
    
    return {'ppg': (ppg_peaks, ppg_peaks_f, ppg_freqs, ppg_fft), 'acc': (acc_peaks, acc_peaks_f, acc_freqs, acc_fft)}
    

In [23]:
def CalConfidence(freqs, fft_mag, bpm_f):
    '''
    This functions calculates the confidence of the heart rate
    Returns:
        Confidence of the estimated heart rate
    '''
    window_f = 30/60
    fundamental_freq_window = (freqs > bpm_f - window_f)&(freqs < bpm_f + window_f)
    return np.sum(fft_mag[fundamental_freq_window])/np.sum(fft_mag)

In [24]:

def estimate(ppg, acc, window_length_s, window_shift_s, fs):
    """
    Estimate the heart rate
    
    Returns:
        The estimated heart rate and the confidence
    """
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    
    est_bpm = []
    confidence = []
    
    prev_est = 40/60
    
    for i in range(0, len(ppg) - window_length, window_shift):
        ppg_window = ppg[i:i+window_length]
        acc_window = acc[i:i+window_length]
        
        window_features = GetFeatures(ppg_window, acc, fs)
        ppg_peaks, ppg_peaks_f, ppg_freqs, ppg_fft = window_features['ppg']
        acc_peaks, acc_peaks_f, acc_freqs, acc_fft = window_features['acc']
        
        ppg_max = ppg_freqs[np.argmax(ppg_fft)]
        acc_max = acc_freqs[np.argmax(acc_fft)]
        
        k = 1
        
        while np.abs(acc_max-ppg_max) <= 0.2 and k <=2:
            k+=1
            ppg_max = ppg_freqs[np.argsort(ppg_fft, axis=0)[-k]]
            acc_max = acc_freqs[np.argsort(acc_fft, axis=0)[-k]]
        
        est_bpm_f = ppg_max
        
        prev_est = est_bpm_f
        est_bpm.append(est_bpm_f*60)
        confidence.append(CalConfidence(ppg_freqs, ppg_fft, est_bpm_f))

    return est_bpm, confidence

In [25]:
def plot_fft(signal, freqs, fft, fs):
    '''
    Plots the fft of the signal
    '''
    plt.subplot(2,1,1)
    ts = np.arange(len(signal)) / fs
    plt.plot(ts, signal)
    plt.subplot(2,1,2)
    plt.plot(freqs, np.abs(fft))
    plt.title("Frequency Domain")
    plt.xlabel("Frequency (Hz)")
    plt.tight_layout()

In [26]:
def RunPulseRateAlgorithm(data_fl, ref_fl):
    '''
    Estimates the pulse rate of the data by using the reference data
    
    Returns:
        error and confidence of the pulse rate calculated
    '''
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # Our data is sampled at 125Hz
    fs = 125
    
    # Bandpass Filter
    ppg = BandPassFilter(ppg, fs)
    accx = BandPassFilter(accx, fs)
    accy = BandPassFilter(accy, fs)
    accz = BandPassFilter(accz, fs)
    
    # Calculate the magnitute of the accelerometers
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # Windowing our ppg and acc signal to estimate
    window_length_s = 8
    window_shift_s = 2    
    
    # Compute pulse rate estimates and estimation confidence.
    est_bpm, confidence = estimate(ppg, acc, window_length_s, window_shift_s, fs)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors = np.abs(np.diag(np.subtract(bpm, est_bpm)))
    
    return errors, confidence

In [27]:
Evaluate()

20.529519533583962

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Code Description :-
1. The code first enters the Evaluate() function. This function loads the Troika data and reference files. Then it runs the pulse rate algorithm to estimate the heart rate.

Data Description :-
1. Data set contains: 
    PPG Signal
    Accelerometer signal in three directions
    
2. ECG signal is used as the reference heart rate(ground truth for our algorithm)

3. The data is quite noisy and we don't have much information about the subjects being collected.

4. Subjects are of age 18-35 years. No information of the gender distribution wasa given.

5. The subject being tested with various activities on the treadmill ( walk, jog, run on treadmill)

Algorithm Description

1. Bandpass filer to make our signal in 40-240bpm range.
2. Compute the aggregate accelerometer magnitude
3. Taking 8s windows , with every window 2s apart of our signal.
4. The estimating the heart rate using the window by running fft on the window signal and finding peaks in frequency domain.
5. Choosing the coorect pulse to estimate heart rate for that window.
6. Compute the confidence of the estimated heart rate
 
Failure mode : The algorithm requires acc, ppg and a reference bpm. If these signals are not present , it will not work.
The algorithm may perform purely if there is a lot of noise in the signal

Caveats : The algorithm uses the accelerometor signal to filter out movement pulse from heart rate.

Algorithm Performance

1. The metric used to compute the error is Mean Absolute Value
2. The estimated heart rate is compared with the reference measurement in the data set.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 